In [1]:
pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()

In [3]:
def read_in_data():
    X_train = pd.read_csv("x_train_sampled_yelp_data.csv")
    y_train = pd.read_csv("y_train_sampled_yelp_data.csv")
    X_test = pd.read_csv("x_test_sampled_yelp_data.csv")
    y_test = pd.read_csv("y_test_sampled_yelp_data.csv")
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = read_in_data()

In [5]:
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", y_test.shape)

X_train shape:  (80000, 2)
Y_train shape:  (80000, 1)
X_test shape:  (20000, 2)
Y_test shape:  (20000, 1)


In [6]:
y_train['stars'].value_counts(normalize=True)

5.0    0.441713
4.0    0.222675
1.0    0.145725
3.0    0.108012
2.0    0.081875
Name: stars, dtype: float64

In [7]:
y_test['stars'].value_counts(normalize=True)

5.0    0.44430
4.0    0.22165
1.0    0.14835
3.0    0.10585
2.0    0.07985
Name: stars, dtype: float64

In [13]:
X_train.head()

,text,date
0,Sunday afternoon savior. They are great! Helpe...,2018-08-06 00:17:29
1,Tuesday was my man and I's weekly wing night. ...,2010-11-11 07:10:55
2,Everything is presented very nicely at Kishimo...,2015-04-29 19:10:30
3,So good we are thinking of having them cater o...,2015-11-13 23:08:28
4,Wow - pleasantly surprised!! Both my son and I...,2011-11-12 01:57:02


In [8]:
#Import pretrained ELMo Embeddings
#elmo = hub.load("https://tfhub.dev/google/elmo/3")
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [9]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([1, 8, 1024])

In [10]:
#Extract ELMo vectors and get mean of the ELMo vectors of constituent terms or tokens

def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [11]:
list_train = [X_train[i:i+100] for i in range(0,X_train.shape[0],100)]
list_test = [X_test[i:i+100] for i in range(0,X_test.shape[0],100)]

In [12]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['text']) for x in list_train]
elmo_test = [elmo_vectors(x['text']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ResourceExhaustedError: OOM when allocating tensor with shape[100,445,44,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node module_apply_default_1/bilm/CNN/Conv2D_6 (defined at /home/jupyter/anaconda3/lib/python3.8/site-packages/tensorflow_hub/native_module.py:615) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.


Original stack trace for 'module_apply_default_1/bilm/CNN/Conv2D_6':
  File "/home/jupyter/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/jupyter/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/jupyter/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jupyter/anaconda3/lib

In [ ]:
#Concatenate back into a single array
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
xtrain = elmo_train_new
ytrain = y_train['stars']
lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(elmo_test_new)
f1_score(y_test['stars'], preds_valid)